$\newcommand{\giraffe}{\text{GiRaFFE}}$
# $\giraffe$: General Relativistic Force-Free Electrodynamics
## Porting the original $\giraffe$ code to NRPy+

Porting the original $\giraffe$ code to NRPy+ will generally make it easier to maintain, as well as to make changes. Specifically, it will make it nearly trivial to increase the finite-differencing order.

Our ultimate goal here will be to code the evolution equation 
\begin{equation}
\partial_t \tilde{S}_i = - \partial_j \left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right) + \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu},
\end{equation}
where the densitized spatial Poynting flux one-form $\tilde{S}_i = \sqrt{\gamma} S_i$ (and $S_i$ comes from $S_{\mu} -n_{\nu} T^{\nu}_{{\rm EM} \mu}$) and
\begin{align}
T^{\mu \nu}_{\rm EM} &= b^2 u^{\mu} u^{\nu} + \frac{b^2}{2} g^{\mu \nu} - b^{\mu} b^{\nu}, \\
\sqrt{4\pi} b^0 &= B^0_{\rm (u)} = \frac{u_j B^j}{\alpha}, \\
\sqrt{4\pi} b^i &= B^i_{\rm (u)} = \frac{B^i + (u_j B^j) u^i}{\alpha u^0}, \\
\end{align}
and 
$$B^i = \frac{\tilde{B}^i}{\gamma}.$$
Furthermore, the four-metric $g_{\mu\nu}$ is related to the three-metric $\gamma_{ij}$, lapse $\beta_i$, and shift $\alpha$ by
$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_i \\
\beta_j & \gamma_{ij}
\end{pmatrix}.
$$
Most of these are computed in the module u0_smallb_Poynting__Cartesian.py, and we will import that module to save effort.
Note that as usual, Greek indices refer to four-dimensional quantities where the zeroth component indicates $t$ components, while Latin indices refer to three-dimensional quantities. Since Python always indexes its lists from 0, however, the zeroth component will indicate a spatial direction, and any expressions involving mixed Greek and Latin indices will need to offset one set of indices by one.

## Preliminaries
First, we will import the core modules of NRPy that we will need. Then, we will declare the gridfunctions related to the metric and build the four metric using code from [Tutorial-smallb2_Poynting_vector-Cartesian.ipynb](Tutorial-smallb2_Poynting_vector-Cartesian.ipynb)

In [1]:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *

#Step 0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Step 1: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)

thismodule = "GiRaFFE_NRPy"


Recall that the four-metric $g_{\mu\nu}$ is related to the three-metric $\gamma_{ij}$, lapse $\beta_i$, and shift $\alpha$ by
$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_i \\
\beta_j & \gamma_{ij}
\end{pmatrix}.
$$
This tensor and its inverse have already been built by the u0_smallb_Poynting__Cartesian.py module, so we can simply import the variables.

In [2]:
# Step 2: Build the four metric
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUX","gammaDD", "sym01",DIM=3)
betaU   = ixp.register_gridfunctions_for_single_rank1("AUX","betaU",DIM=3)
alpha   = gri.register_gridfunctions("AUX","alpha")
ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUX","ValenciavU",DIM=3)
BtildeU = ixp.register_gridfunctions_for_single_rank1("AUX","BtildeU",DIM=3)

import u0_smallb_Poynting__Cartesian.u0_smallb_Poynting__Cartesian as u0b
u0b.compute_u0_smallb_Poynting__Cartesian(gammaDD,betaU,alpha,ValenciavU,BtildeU)

dummygammaUU, gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)
betaD = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        betaD[i] = gammaDD[i][j] * betaU[j]

# We will now pull in the four metric and its inverse.
g4DD = ixp.zerorank2(DIM=4)
g4UU = ixp.zerorank2(DIM=4)
for mu in range(4):
    for nu in range(4):
        g4DD[mu][nu] = u0b.g4DD[mu][nu]
        g4UU[mu][nu] = u0b.g4UU[mu][nu]


We will also need spatial derivatives of the metric, $\partial_i g_{\mu\nu} = g_{\mu\nu,i}$. In terms of the three-metric, lapse, and shift, we find
$$
g_{\mu\nu,i} = \begin{pmatrix} 
-2\alpha \alpha_{,i} + \beta^k_{\ ,i} \beta_k + \beta^k \beta_{k,i} & \beta_{i,i} \\
\beta_{j,i} & \gamma_{ij,i}
\end{pmatrix}.
$$

Since this expression mixes Greek and Latin indices, we will need to store the expressions for each of the three spatial derivatives as separate variables. 
Also, consider the term $\beta_{i,j} = \partial_j \beta_i = \partial_j (\gamma_{ik} \beta^k) =  \gamma_{ik} \partial_j\beta^k + \beta^k \partial_j \gamma_{ik} = \gamma_{ik} \beta^k_{\ ,j} + \beta^k \gamma_{ik,j}$

In [3]:
# Step 3: Build the spatial derivative of the four metric
# Step 3a: Declare derivatives of grid functions. These will be handled by FD_outputC
alpha_dD   = ixp.declarerank1("alpha_dD")
betaU_dD   = ixp.declarerank2("betaU_dD","nosym")
gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym01")

# Step 3b: These derivatives will be constructed analytically.
betaDdD    = ixp.zerorank2()

# Zach says: Just set this up as a rank3 tensor with DIM=4. We'll not set the time deriv. 
#  This will get rid of a lot of unnecessary code below.
g4DDdD     = ixp.zerorank3(DIM=4)

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            betaDdD[i][j] = gammaDD[i][k] * betaU_dD[k][j] + betaU[k] * gammaDD_dD[i][k][j]

# Step 3c: Set the 00 components
for i in range(1,4):
    g4DDdD[0][0][i] = -2*alpha*alpha_dD[i-1]
    for k in range(DIM):
        g4DDdD[0][0][i] += betaU_dD[k][i-1] * betaD[k] + betaU[k] * betaDdD[k][i-1]
    for mu in range(1,4):
        g4DDdD[mu][0][i] = g4DDdD[0][mu][i-1] = betaDdD[mu-1][i-1]
    for mu in range(1,4):
        for nu in range(1,4):
            g4DDdD[mu][nu][i] = gammaDD_dD[mu-1][nu-1][i-1]
        

Now that the metric and its derivatives are out of the way, we will return our attention to the electromagnetic stress-energy tensor
$$T^{\mu \nu}_{\rm EM} = b^2 u^{\mu} u^{\nu} + \frac{b^2}{2} g^{\mu \nu} - b^{\mu} b^{\nu}.$$
We will need the four-velocity $u^\mu$, where 
\begin{align}
u^i &= u^0 (\alpha v^i_{(n)} - \beta^i), \\
u_j &= \alpha u^0 \gamma_{ij} v^i_{(n)}, \\
\end{align}
and $v^i_{(n)}$ is the Valencia three-velocity. The values of $u^0$ and $v^i_{(n)}$ can be read from HydroBase and IllinoisGRMHD. These have already been built by the u0_smallb_Poynting__Cartesian.py module, so we can simply import the variables.


In [4]:
u0 = par.Cparameters("REAL",thismodule,"u0")
uD = ixp.zerorank1()

for i in range(DIM):
    for j in range(DIM):
        uD[j] += alpha*u0*gammaDD[i][j]*ValenciavU[j]

uU = ixp.zerorank1()
for i in range(DIM):
    uU[i] = u0*(alpha*ValenciavU[i] - betaU[i])


We also need the vector $b^{\mu}$ before we can compute this, which is related to the magnetic field by 
\begin{align}
b^0 &= \frac{1}{\sqrt{4\pi}} B^0_{\rm (u)} = \frac{u_j B^j}{\sqrt{4\pi}\alpha}, \\
b^i &= \frac{1}{\sqrt{4\pi}} B^i_{\rm (u)} = \frac{B^i + (u_j B^j) u^i}{\sqrt{4\pi}\alpha u^0}, \\
\end{align} and \begin{align}
B^i &= \frac{\tilde{B}^i}{\gamma},
\end{align}
where $\tilde{B}^i$ is the variable tracked by the HydroBase thorn in the Einstein Toolkit. These have already been built by the u0_smallb_Poynting__Cartesian.py module, so we can simply import the variables.

In [5]:
smallbU = ixp.zerorank1(DIM=4)
for mu in range(4):
    smallbU[mu] = u0b.smallb4U[mu]

smallb2 = u0b.smallb2


We now have all the pieces to calculate the stress-energy tensor,
$$T^{\mu \nu}_{\rm EM} = b^2 u^{\mu} u^{\nu} + \frac{b^2}{2} g^{\mu \nu} - b^{\mu} b^{\nu}.$$


In [6]:
TEMUU = ixp.register_gridfunctions_for_single_rank2("AUX","TEMUU","sym01",DIM=4)

TEMUU[0][0] = smallb2*u0*u0 + smallb2*g4UU[0][0]/2 - smallbU[0]*smallbU[0]
for mu in range(1,4):
    TEMUU[mu][0] = TEMUU[0][mu] = smallb2*uU[mu-1]*u0 + smallb2*g4UU[mu][0]/2 - smallbU[mu]*smallbU[0]
for mu in range(1,4):
    for nu in range(1,4):
        TEMUU[mu][nu] = smallb2*uU[mu-1]*uU[nu-1] + smallb2*g4UU[mu][nu]/2 - smallbU[mu]*smallbU[nu]

If we look at the evolution equation, we see that we will need spatial  derivatives of $T^{\mu\nu}_{EM}$; analytically expanding that derivative would be a rather tedious affair, so to avoid it, we will simply declare the parenthetical term $$\left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right)$$ as a gridfunction, and set it accordingly. Then, we declare derivatives of it.

In [7]:
# We already handled the ADMBase variables' derivatives when we built g4DDdD.
# That leaves the valencia 3 velocity and tilde B field.
alpsqrtgamTEMUD = ixp.register_gridfunctions_for_single_rank2("AUX","alpsqrtgamTEMUD","nosym")
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            # Since TEMUU is a 4D quantity, we increment its indices to match gamma
            alpsqrtgamTEMUD[i][j] = alpha * sp.sqrt(gammaDET) * gammaDD[k][i] * TEMUU[j+1][k+1]

alpsqrtgamTEMUD_dD = ixp.declarerank3("alpsqrtgamTEMUD","nosym")

Finally, we will return our attention to the time evolution equation,
\begin{equation}
\partial_t \tilde{S}_i = - \partial_j \left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right) + \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}.
\end{equation}
We first construct the second term, to reduce the complication of dealing with mixed Greek and Latin indices.

In [8]:
secondterm = ixp.zerorank1()
for i in range(DIM):
    for mu in range(DIM):
        for nu in range(DIM):
            secondterm[i] += alpha * sp.sqrt(gammaDET) * TEMUU[mu][nu] * g4DDdD[mu][nu][i] / 2

Stilde_rhs = ixp.register_gridfunctions_for_single_rank1("EVOL","Stilde_rhs")
for i in range(DIM):
    Stilde_rhs[i] = secondterm[i]
    for j in range(DIM):
        Stilde_rhs[i] -= alpsqrtgamTEMUD_dD[j][i][j]